# 💾  Monitor API endpoint and log predictions in a Rubrix dataset

In this tutorial, you'll learn to monitoring the predictions of a FastAPI inference endpoint
and log predictions in a Rubrix dataset. 


This tutorial walks you through 4 basic steps to automate your endpoint:

- 💾 Load the model you want to use.
- 🔄 Convert model output to Rubrix format.
- 💻 Create a FastAPI endpoint.
- 🤖 Add middleware to automate logging to Rubrix


Let's get started!


![Rubrix Log Demo](./img/automatic_fastapi_log/automatic_fastapi_log.gif)

## Setup Rubrix

**If you are new to Rubrix, visit and ⭐ star Rubrix for more materials like and detailed docs**: [Github repo](https://github.com/recognai/rubrix)

If you have not installed and launched Rubrix, check the [Setup and Installation guide](https://docs.rubrix.ml/en/latest/getting_started/setup%26installation.html).

## Install tutorial dependencies

Apart from Rubrix, we'll need the `transformers` and `uvicorn` libraries and the `distilbert-base-uncased-finetuned-sst-2-english` sentiment-analysis model. If you do not have the libraries, run:

In [ ]:
!pip install transformers
!pip install uvicorn[standard]

The model will be downloaded in the next step.

## Loading model from HuggingFace Transformers

Let's get and load our model pretrained pipeline and apply it to one of our dataset records:

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", return_all_scores=True)

For more informations about using the `transformers` library, check the "Explore data and predictions with `datasets` and `transformers`" [tutorial](https://rubrix.readthedocs.io/en/stable/tutorials/01-huggingface.html).

### Model output
Let's try the model in this example:

In [ ]:
from pprint import pprint

batch = ['I really like rubrix!']
predictions = classifier(batch)
pprint(predictions)

Looks like the `predictions` is a list containing lists of two elements : 
- The first dictionnary containing the `NEGATIVE` sentiment label and its score.
- The second dictionnary containing the same data but for `POSITIVE` sentiment.

## Convert output to Rubrix format
To log the output to rubrix we should supply a list of dictionnaries, each dictonnary containing two keys:
- `labels` : value is a list of strings, each string being the label of the sentiment.
- `probabilities` : value is a list of floats, each float being the probability of the sentiment.

In [ ]:
rubrix_format = [
    {
        "labels": [p["label"] for p in prediction],
        "probabilities": [p["score"] for p in prediction],
    }
    for prediction in predictions
]
pprint(rubrix_format)

## Create prediction endpoint

In [ ]:
from fastapi import FastAPI
from typing import List

app = FastAPI()

@app.post("/predict")
def predict_batch(batch: List[str]):
    predictions = classifier(batch)
    return [
        {
            "labels": [p["label"] for p in prediction],
            "probabilities": [p["score"] for p in prediction],
        }
        for prediction in predictions
    ]

## Add Rubrix logging middleware to the application

In [ ]:
from rubrix.client.asgi import RubrixLogHTTPMiddleware

app.add_middleware(
    RubrixLogHTTPMiddleware,
    api_endpoint="/predict", #the endpoint that will be logged
    dataset="monitoring_dataset_v1", #your dataset name
    # you could post-process the predict output with a custom record_mapper function
    # record_mapper=custom_text_classification_mapper,
)

### Launch the appplication

To launch the application, copy the whole code into a file named `main.py` and run the following command:

In [ ]:
!uvicorn main:app - -reload

![Rubrix Log Demo](./img/automatic_fastapi_log/automatic_fastapi_log.gif)

## Summary
In this tutorial, we have learnt to automatically log model outputs into Rubrix, this can be used to build FastAPI applications faster and without worrying about interacting with Rubrix.

## Next steps

### 📚 [Rubrix documentation](https://docs.rubrix.ml) for more guides and tutorials.

### 🙋‍♀️ Join the Rubrix community! A good place to start is the [discussion forum](https://github.com/recognai/rubrix/discussions).

### ⭐ Rubrix [Github repo](https://github.com/recognai/rubrix) to stay updated.